In [ ]:
"""
Created on Fri Feb 15 11:43 2021

use script to identify ice shelves in NEMO data from Christoph
=> updated to use the corrected ice shelf draft and the ice shelf concentration
# to be used from now on! (at least for non-huge data)

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
#import matplotlib.pyplot as plt
from pyproj import Transformer
#from matplotlib import cm
#import cartopy
#import cartopy.crs as ccrs
#import matplotlib as mpl
import pandas as pd
from tqdm.notebook import trange, tqdm
#from tqdm import tqdm
import multimelt.plume_functions as pf
import basal_melt_param.box_functions as bf
import basal_melt_param.useful_functions as uf
import multimelt.create_isf_mask_functions as isfmf

from IPython.display import clear_output

In [ ]:
%matplotlib qt5

In [ ]:
map_lim = [-3000000,3000000]

#chunk_size = 700
chunk_size = False

In [ ]:
nemo_run = 'isfru94' # 'isf94','isfru94','ctrl94'

In [ ]:
######
###### READ IN DATA
######


#if run on luke
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
outputpath_boxes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/nemo_5km_'+nemo_run+'/'


#ds_nemo = xr.open_dataset(outputpath_mask_orig+'nemo_5km_isf_masks_and_info_and_distance.nc')

Create the masks for ice shelves/ground/pinning points/grounding line

WITH FIXED GEOMETRY

In [ ]:
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_mask_cut = uf.cut_domain_stereo(file_mask, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

#file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_'+str(timet.values.astype(int))+'.nc').isel(time=0).drop('time')
#file_TS_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim)

file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft']
file_msk = file_mask_cut['ls_mask012']#.where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_mask_cut['x']
yy = file_mask_cut['y']

whole_ds_tt = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                          inputpath_metadata+'lonlat_masks.txt', outputpath_mask, 
                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=False, variable_geometry=False,
                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                          write_metadata='yes')

print('------- WRITE TO NETCDF -----------')
whole_ds_tt.to_netcdf(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc','w')

Prepare the box characteristics (writes the output directly to files)

In [ ]:
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_mask_cut = uf.cut_domain_stereo(file_mask, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)          

file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft']
file_msk = file_mask_cut['ls_mask012']#.where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_mask_cut['x']
yy = file_mask_cut['y']

whole_ds_tt = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')

nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
if 'labels' in file_isf.coords.keys():
    file_isf = file_isf.drop('labels')

In [ ]:
isf_var_of_int = whole_ds_tt[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'],isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)
out_2D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
out_1D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
    
    

In [ ]:

box_1D = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_1970.nc')
box_2D = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_1970.nc')

In [ ]:
isf_var_of_int = whole_ds[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'],isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)
#out_2D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_2D.nc') # separated Filchner and Ronne (before review)
#out_1D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_1D.nc') # separated Filchner and Ronne (before review)
out_2D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
out_1D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')

Prepare the plume characteristics

In [ ]:
plume_param_options = ['simple','lazero', 'appenB']

file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)

file_draft = file_other_cut['corrected_isfdraft']

xx = file_mask_cut['x']
yy = file_mask_cut['y']

whole_ds_tt = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')

nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
if 'labels' in file_isf.coords.keys():
    file_isf = file_isf.drop('labels')

plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

# Compute the ice draft
ice_draft_pos = file_draft
# Be careful with ice shelf 178 and 195 - they have a negative ice draft
# I don't know how to fix it at the moment so I put it to nan
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 178, np.nan)
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 195, np.nan)

ice_draft_neg = -1*ice_draft_pos


plume_charac = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int)
print('------ WRITE TO NETCDF -------')
#plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics.nc')
plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')

In [ ]:
plume_param_options = ['new_lazero'] #'cavity',, 'local','lazero'

plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

# Compute the ice draft
ice_draft_pos = file_draft
# Be careful with ice shelf 178 and 195 - they have a negative ice draft
# I don't know how to fix it at the moment so I put it to nan
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 178, np.nan)
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 195, np.nan)

ice_draft_neg = -1*ice_draft_pos

plume_charac_shift2 = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int, grad_corr=5, dir_nb=16, extra_shift=2, dist_incl=1)
plume_charac_shift1 = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int, grad_corr=5, dir_nb=16, extra_shift=1, dist_incl=1)
plume_charac = plume_charac_shift2.combine_first(plume_charac_shift1)
print('------ WRITE TO NETCDF -------')
plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_lazero_comparison.nc')


In [ ]:
plume_charac_old = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')
plume_charac_new = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_lazero_comparison.nc')


In [ ]:
((plume_charac_old['alpha'].sel(option='lazero') == 0) & (file_isf['ISF_mask'] == 42)).plot()

In [ ]:
plt.figure()
((np.isnan(plume_charac_new['zGL'].sel(option='new_lazero'))) & (file_isf['ISF_mask'] == 42)).plot()

In [ ]:
plt.figure()
plume_charac_new['zGL'].sel(option='new_lazero').plot() #.where(file_isf['ISF_mask'] == 42

FILL POINTS WHERE SLOPE IS NOT INCLINED TOWARDS GROUNDING LINE

In [ ]:
plume_charac = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')
plume_charac = plume_charac.sel(y=plume_charac.y[::-1])

file_isf_new = file_isf.sel(y=file_isf.y[::-1])

alpha_all_corrected = plume_charac['alpha'].sel(option='lazero') * np.nan
zGL_all_corrected = plume_charac['zGL'].sel(option='lazero') * np.nan


for kisf in tqdm(file_isf.Nisf):
    alpha_kisf_lazero = plume_charac['alpha'].sel(option='lazero').where(file_isf_new['ISF_mask']==kisf, drop=True).drop('Nisf')
    alpha_kisf_local = plume_charac['alpha'].sel(option='appenB').where(file_isf_new['ISF_mask']==kisf, drop=True).drop('Nisf')
    alpha_kisf_lazero_corrected = alpha_kisf_lazero.where(alpha_kisf_lazero > 0, alpha_kisf_local)

    zGL_kisf_lazero = plume_charac['zGL'].sel(option='lazero').where(file_isf_new['ISF_mask']==kisf, drop=True).drop('Nisf')
    zGL_kisf_local = plume_charac['zGL'].sel(option='appenB').where(file_isf_new['ISF_mask']==kisf, drop=True).drop('Nisf')

    zGL_kisf_lazero_fillx = zGL_kisf_lazero.where(alpha_kisf_lazero != 0).interpolate_na(dim='x',method='nearest')
    zGL_kisf_lazero_filly = zGL_kisf_lazero.where(alpha_kisf_lazero != 0).interpolate_na(dim='y',method='nearest')
    zGL_kisf_lazero_maxboth = xr.concat([zGL_kisf_lazero_fillx,zGL_kisf_lazero_filly],dim='dim0').max('dim0')
    #zGL_kisf_lazero_999_minboth = xr.concat([zGL_kisf_lazero_999_fillx,zGL_kisf_lazero_999_filly],dim='dim0').mean('dim0')
    zGL_kisf_lazero_corrected = zGL_kisf_lazero_maxboth.where(np.isfinite(zGL_kisf_lazero))
    
    alpha_all_corrected = alpha_all_corrected.where(file_isf_new['ISF_mask']!=kisf, alpha_kisf_lazero_corrected.reindex_like(alpha_all_corrected))
    zGL_all_corrected = zGL_all_corrected.where(file_isf_new['ISF_mask']!=kisf, zGL_kisf_lazero_corrected.reindex_like(alpha_all_corrected))
alpha_all_corrected = alpha_all_corrected.where(np.isfinite(alpha_all_corrected), 0).drop('Nisf')
zGL_all_corrected = zGL_all_corrected.where(np.isfinite(alpha_all_corrected), 0).drop('Nisf')

In [ ]:
plume_charac['alpha'] = xr.concat([plume_charac['alpha'].sel(option='simple'),alpha_all_corrected,plume_charac['alpha'].sel(option='appenB')], dim='option', coords='minimal').sel(y=plume_charac.y[::-1])
plume_charac['zGL'] = xr.concat([plume_charac['zGL'].sel(option='simple'),zGL_all_corrected,plume_charac['zGL'].sel(option='appenB')], dim='option', coords='minimal').sel(y=plume_charac.y[::-1])


In [ ]:
print('------ WRITE TO NETCDF -------')
plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_corrected.nc')

Prepare correct bathymetry (accounting for ice shelf concentration but also if we are at ice front or grounding line)

In [ ]:
whole_ds = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')

file_bed_orig = file_mask_orig_cut['bathy_metry']
file_bed_corr = file_other_cut['corrected_isf_bathy']
file_draft = file_other_cut['corrected_isfdraft'] 

file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)
file_bed_goodGL_with_ocean =  file_bed_goodGL.where(whole_ds['ISF_mask'] > 1, file_bed_orig)
file_bed_goodGL_with_ocean.to_dataset(name='bathymetry').to_netcdf(outputpath_mask + 'processed_bathymetry.nc')

CHECK RESULTS

In [ ]:
out_2D_boxes = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
out_1D_boxes = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')

In [ ]:
out_2D_boxes['box_location'].sel(box_nb_tot=out_1D_boxes['nD_config'].sel(config=4,Nisf=75)).plot()

In [ ]:
whole_ds = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')


In [ ]:
nonnan_Nisf = whole_ds['Nisf'].where(np.isfinite(whole_ds['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask'] == file_isf.Nisf).sum('Nisf')
isf_mask = isf_mask.where(isf_mask)

In [ ]:
GL_flux = file_isf['GL_flux']

In [ ]:
GL_flux_new = GL_flux.where(isf_mask==file_isf.Nisf).sum('Nisf')
GL_flux_new = GL_flux_new.where(GL_flux_new)

In [ ]:
GL_flux_new.plot()

In [ ]:
nisf_list = whole_ds.Nisf.values.tolist()
nisf_list.remove(11)
nisf_list.remove(10)

In [ ]:
nisf_list

In [ ]:
whole_ds = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_new.nc')
plume_charac = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics.nc')

In [ ]:
plume_charac['alpha'].sel(option='simple').where(whole_ds['ISF_mask']==75,drop=True).min()

In [ ]:
ice_draft_pos.where(whole_ds['ISF_mask']==75, drop=True).plot()

In [ ]:
plume_var_of_int['GL_mask'].plot()

In [ ]:
plume_var_of_int['GL_mask'].where(plume_var_of_int['GL_mask']==75, drop=True).plot()

In [ ]:
whole_ds['ISF_mask'].where(whole_ds['ISF_mask']==0).plot()

In [ ]:
dx = region_isf.x[1] - region_isf.x[0]
dy = region_isf.y[1] - region_isf.y[0]

In [ ]:
region_isf = whole_ds['ISF_mask'].where(whole_ds['ISF_mask']==75, drop=True)
region_x = np.arange(region_isf.x.min() - 2*dx, region_isf.x.max() + 2.5*dx, dx)
region_y = np.arange(region_isf.y.max() - 2*dy, region_isf.y.min() + 2.5*dy, dy)


In [ ]:
region_to_look_at = whole_ds['ISF_mask'].sel(x=region_x,y=region_y)
mask_ground = region_to_look_at.where(region_to_look_at != 0, 2)

In [ ]:
mask_10 = mask_ground.where(mask_ground==2, 0) * 0.5

In [ ]:
mask_10.plot()

In [ ]:
weights_neighbors = np.array(([0, 1, 0], [1, 1, 1], [0, 1, 0]))
xr_weights = xr.DataArray(data=weights_neighbors, dims=['y', 'x'])

xr_corr_neighbors = mask_10.copy(data=pf.nd_corr(mask_10,xr_weights))

cut_gline = xr_corr_neighbors.where((region_to_look_at>1) & (xr_corr_neighbors>0))
mask_gline = region_to_look_at.where(cut_gline>0)

In [ ]:
mask_gline.plot()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
region_to_look_at.plot()

In [ ]:
whole_ds['ISF_mask'].y

In [ ]:
plt.figure()
whole_ds['ISF_mask'].sel(x=np.arange(-2998000.,0.5,dx),y=np.arange(2998000.,0,dy)).plot(vmax=2)

In [ ]:
add_fac=150

In [ ]:
mask_gline_orig = whole_ds['GL_mask']

###################

larger_region = whole_ds['ISF_mask'].sel(x=np.arange(-2998000.,0.5,dx),y=np.arange(2998000.,0,dy))
mask_10_first = larger_region.where(larger_region == 0, 5).where(larger_region != 0, 1)
mask_gnd = mask_10_first.where(mask_10_first == 1, 0) #set all ice shelves and open ocean to 0, set all grounded ice to 1

meshx_gnd, meshy_gnd = np.meshgrid(mask_gnd.x,mask_gnd.y)
meshx_gnd_da = mask_gnd.copy(data=meshx_gnd)
meshy_gnd_da = mask_gnd.copy(data=meshy_gnd)

core = mask_gnd.sel(x=np.arange(-1938000.,-1900000., dx),y=np.arange(718000.,680000., dy)).reindex_like(mask_gnd)
mask_core = mask_gnd.where(np.isnan(core),5)

# filter that checks the point around
weights_filter = np.zeros((3,3))
weights_filter[0,1] = 1
weights_filter[1,0] = 1
weights_filter[1,2] = 1
weights_filter[2,1] = 1

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

iter_mask = mask_core.copy()
for n in tqdm(range(add_fac)):
    corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
    iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

mask_ground = iter_mask.where(iter_mask !=5, 2)#.reindex_like(la)
mask_ground = mask_ground.where(mask_ground>0,0)

##########################################

mask_10 = mask_ground.where(mask_ground==2, 0) * 0.5
weights_neighbors = np.array(([0, 1, 0], [1, 1, 1], [0, 1, 0]))
xr_weights = xr.DataArray(data=weights_neighbors, dims=['y', 'x'])

xr_corr_neighbors = mask_10.copy(data=pf.nd_corr(mask_10,xr_weights))

cut_gline = xr_corr_neighbors.where((larger_region>1) & (xr_corr_neighbors>0))
mask_gline = larger_region.where(cut_gline>0).reindex_like(mask_gline_orig)

mask_gline_final = mask_gline_orig.where(mask_gline != 9, mask_gline)
mask_gline_final = mask_gline_final.where(mask_gline != 54, mask_gline)
mask_gline_final = mask_gline_final.where(mask_gline != 75, mask_gline)
mask_gline_final = mask_gline_final.where(mask_gline != 98, mask_gline)
mask_gline_final = mask_gline_final.where(mask_gline != 99, mask_gline)
mask_gline_final = mask_gline_final.where(mask_gline != 100, mask_gline)

In [ ]:
whole_ds['isf_name'].sel(Nisf=9)

In [ ]:
mask_gline_final.plot()

In [ ]:
whole_ds['ISF_mask'].y.sel(y=720000, method='nearest')

In [ ]:
mask_10 = file_msk.where(file_msk == 2, 0).where(file_msk != 2,1) #set all ice shelves and open ocean to 0, set all grounded ice to 1

mask_gnd = mask_10.where(mask_10>0, drop=True)
mask_gnd = mask_gnd.where(mask_gnd>0,0)

meshx_gnd, meshy_gnd = np.meshgrid(mask_gnd.x,mask_gnd.y)
meshx_gnd_da = mask_gnd.copy(data=meshx_gnd)
meshy_gnd_da = mask_gnd.copy(data=meshy_gnd)


max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
half_range = round(max_len_xy/2)

mask_core = mask_gnd.where(~((uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]) # assuming that the South Pole is in the center of the projection
                      & uf.in_range(meshy_gnd_da, [-2*dist*dy,2*dist*dy]))), 5)

# filter that checks the point around
weights_filter = np.zeros((3,3))
weights_filter[0,1] = 1
weights_filter[1,0] = 1
weights_filter[1,2] = 1
weights_filter[2,1] = 1

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

iter_mask = mask_core.copy()
for n in tqdm(range(half_range+2*dist+add_fac)):
    corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
    iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)
mask_ground = mask_ground.where(mask_ground>0,0)